# Languages

This tutorial will explain how to set the `language` property for various nodes and file objects when using the `ricecooker` framework.

## Import libraries

  - We create a sushi chef class by subclassing `ricecooker.chefs.SushiChef`.
  - The languages supported by Kolibri and the Content Curation Server are provided in `le_utils.constants.languages`.
  - We'll also use `TopicNode` (folders) and `DocumentNode` (PDFs) nodes,
    and a `DocumentFile` (PDF)
  - We'll also need the constants in `licenses` to set node's licenses (required attribute)


In [1]:
from ricecooker.chefs import SushiChef
from le_utils.constants import languages as languages
from ricecooker.classes.nodes import ChannelNode, TopicNode, DocumentNode, VideoNode
from ricecooker.classes.files import DocumentFile
from le_utils.constants import licenses

## Explore language objects and language codes

In [2]:
# can lookup language using language code
language_obj = languages.getlang('en')
language_obj

Language(native_name='English', primary_code='en', subcode=None, name='English', ka_name=None)

In [3]:
# can lookup language using language name (the new le_utils version has not shipped yet)
language_obj = languages.getlang_by_name('English')
language_obj

Language(native_name='English', primary_code='en', subcode=None, name='English', ka_name=None)

In [4]:
# all `language` attributed (channel, nodes, and files) need to use language code
language_obj.code

'en'

The above language code is an internal representaiton that uses two-letter codes, and sometimes has a locale information, e.g., `pt-BR` for Brazilian Portuiguese. Sometimes the internal code representaiton for a language is the three-letter vesion, e.g., `zul` for Zulu.

## Create chef class

We now create subclass of `ricecooker.chefs.SushiChef` and defined its `get_channel` and `construct_channel` methods.

For the purpose of this example, we'll create three topic nodes in different languages that contain one document in each.

In [5]:
class MySushiChef(SushiChef):
    """
    A sushi chef that creates a channel with content in EN, FR, and SP.
    """
    def get_channel(self, **kwargs):
        channel = ChannelNode(
            source_domain='testing.org',
            source_id='lang_test_chanl',
            title='Languages test channel',
            thumbnail='http://themes.mysitemyway.com/_shared/images/flags.png',
            language = languages.getlang('en').code   # set global language for channel (will apply as default option to all content items in this channel)
        )
        return channel

    def construct_channel(self, **kwargs):
        # create channel
        channel = self.get_channel(**kwargs)

        # create the English topic, add a DocumentNode to it
        topic = TopicNode(
            source_id="<en_topic_id>",
            title="New Topic in English",
            language=languages.getlang('en').code,
        )
        doc_node = DocumentNode(
            source_id="<en_doc_id>",
            title='Some doc in English',
            description='This is a sample document node in English',
            files=[DocumentFile(path='samplefiles/documents/doc_EN.pdf')],
            license=licenses.PUBLIC_DOMAIN,
            language=languages.getlang('en').code,
        )
        topic.add_child(doc_node)
        channel.add_child(topic)

        # create the Spanish topic, add a DocumentNode to it
        topic = TopicNode(
            source_id="<es_topic_id>",
            title="Topic in Spanish",
            language=languages.getlang('es-MX').code,
        )
        doc_node = DocumentNode(
            source_id="<es_doc_id>",
            title='Some doc in Spanish',
            description='This is a sample document node in Spanish',
            files=[DocumentFile(path='samplefiles/documents/doc_ES.pdf')],
            license=licenses.PUBLIC_DOMAIN,
            language=languages.getlang('es-MX').code,
        )
        topic.add_child(doc_node)
        channel.add_child(topic)

        # create the French topic, add a DocumentNode to it
        topic = TopicNode(
            source_id="<fr_topic_id>",
            title="Topic in French",
            language=languages.getlang('fr').code,
        )
        doc_node = DocumentNode(
            source_id="<fr_doc_id>",
            title='Some doc in English',
            description='This is a sample document node in French',
            files=[DocumentFile(path='samplefiles/documents/doc_FR.pdf')],
            license=licenses.PUBLIC_DOMAIN,
            language=languages.getlang('fr').code,
        )
        topic.add_child(doc_node)
        channel.add_child(topic)

        return channel


Run of you chef by creating an instance of the chef class and calling it's `run` method:

In [6]:
mychef = MySushiChef()
args = {'token': 'YOURTOKENHERE9139139f3a23232', 'reset': True, 'verbose': True, 'publish': True}
options = {}
mychef.run(args, options)

Logged in with username you@yourdomain.org
You are using Ricecooker v0.6.2, however v0.6.3 is available. You should consider upgrading your Ricecooker.
Running get_channel... 
run_id: f0427e13c6ba48e593155a204c6dfaab


***** Starting channel build process *****




WebSocket closed, retrying send.
WebSocket closed, retrying send.
WebSocket closed, retrying send.


Calling construct_channel... 
   Setting up initial channel structure... 
   Validating channel structure...
      Languages test channel (ChannelNode): 6 descendants
         New Topic in English (TopicNode): 1 descendant
            Some doc in English (DocumentNode): 1 file
         Topic in Spanish (TopicNode): 1 descendant
            Some doc in Spanish (DocumentNode): 1 file
         Topic in French (TopicNode): 1 descendant
            Some doc in English (DocumentNode): 1 file
   Tree is valid

Processing content...
	--- Downloaded e8b1fe37ce3da500241b4af4e018a2d7.pdf
	--- Downloaded cef22cce0e1d3ba08861fc97476b8ccf.pdf
	--- Downloaded 6c8730e3e2554e6eac0ad79304bbcc68.pdf
	--- Downloaded de498249b8d4395a4ef9db17ec02dc91.png


   All files were successfully downloaded

Checking if files exist on Kolibri Studio...


Getting file diff...


...


Creating tree on Kolibri Studio...
   Creating channel Languages test channel


Creating channel...


	Preparing fields...
(0 of 6 uploaded)    Processing Languages test channel (ChannelNode)
(3 of 6 uploaded)       Processing New Topic in English (TopicNode)
(4 of 6 uploaded)       Processing Topic in Spanish (TopicNode)
(5 of 6 uploaded)       Processing Topic in French (TopicNode)
   All nodes were created successfully.
Upload time: 4.160898s

Publishing tree to Kolibri... 


Publishing channel...




DONE: Channel created at https://contentworkshop.learningequality.org/channels/cba91822d3ab5a748cd19532661d690f/edit



Congratulations, you put three languages on the internet!

## Example 2: YouTube video with subtitles in multiple languages

You can use the library `youtube_dl` to get lots of useful metadata about videos and playlists, including the which language subtitle are vailable for a video.


In [7]:
import youtube_dl

ydl = youtube_dl.YoutubeDL({
    'quiet': True,
    'no_warnings': True,
    'writesubtitles': True,
    'allsubtitles': True,
})


youtube_id =  'FN12ty5ztAs'

info = ydl.extract_info(youtube_id, download=False)
subtitle_languages = info["subtitles"].keys()

print(subtitle_languages)

dict_keys(['en', 'fr', 'zu'])


### Full sushi chef example

The `YoutubeVideoWithSubtitlesSushiChef` class below shows how to create a channel with youtube video and upload subtitles files with all available languages.

In [11]:
from ricecooker.chefs import SushiChef
from ricecooker.classes import licenses
from ricecooker.classes.nodes import ChannelNode, TopicNode, VideoNode
from ricecooker.classes.files import YouTubeVideoFile, YouTubeSubtitleFile


import youtube_dl
ydl = youtube_dl.YoutubeDL({
    'quiet': True,
    'no_warnings': True,
    'writesubtitles': True,
    'allsubtitles': True,
})


class YoutubeVideoWithSubtitlesSushiChef(SushiChef):
    """
    A sushi chef that creates a channel with content in EN, FR, and SP.
    """
    channel_info = {
        'CHANNEL_SOURCE_DOMAIN': 'learningequality.org',       # make sure to change this when testing
        'CHANNEL_SOURCE_ID': 'youtube_vid_with_subs',   # channel's unique id
        'CHANNEL_TITLE': 'Youtube subtitles downloading chef',
        'CHANNEL_LANGUAGE': 'en',
        'CHANNEL_THUMBNAIL': 'http://themes.mysitemyway.com/_shared/images/flags.png',
        'CHANNEL_DESCRIPTION': 'This is a test channel to make sure youtube subtitle languages lookup works'
    }

    def construct_channel(self, **kwargs):
        # create channel
        channel = self.get_channel(**kwargs)

        # get all subtitles available for a sample video
        youtube_id =  'FN12ty5ztAs'
        info = ydl.extract_info(youtube_id, download=False)
        subtitle_languages = info["subtitles"].keys()
        print('Found subtitle_languages = ', subtitle_languages)
        
        # create video node
        video_node = VideoNode(
            source_id=youtube_id,
            title='Youtube video',
            license=licenses.SpecialPermissionsLicense(description="Permission granted by Touchable Earth to distribute through Kolibri."),
            derive_thumbnail=True,
            files=[YouTubeVideoFile(youtube_id=youtube_id)],
        )

        # add subtitles in whichever languages are available.
        for lang_code in subtitle_languages:
            video_node.add_file(
                YouTubeSubtitleFile(
                    youtube_id=youtube_id,
                    language=lang_code
                )
            )

        channel.add_child(video_node)

        return channel

    

In [12]:
chef = YoutubeVideoWithSubtitlesSushiChef()
args = {'token': 'YOURTOKENHERE9139139f3a23232', 'reset': True, 'verbose': True}
options = {}
chef.run(args, options)

Logged in with username you@yourdomain.org
You are using Ricecooker v0.6.2, however v0.6.3 is available. You should consider upgrading your Ricecooker.
Running get_channel... 
run_id: 2d41f6af69da495aa719b3b069e6b2aa


***** Starting channel build process *****




WebSocket closed, retrying send.
WebSocket closed, retrying send.
WebSocket closed, retrying send.


Calling construct_channel... 


Found subtitle_languages =  dict_keys(['en', 'fr', 'zu'])


   Setting up initial channel structure... 
   Validating channel structure...
      Youtube subtitles downloading chef (ChannelNode): 1 descendant
         Youtube video (VideoNode): 4 files
   Tree is valid

Processing content...
	--- Downloaded (YouTube) f9237acf3dadfbc63079a615c8cc944c.mp4
	--- Downloaded subtitle a94e1e3702d1f182db1a054feb68f147.vtt
	--- Downloaded subtitle 99d24a5240d64e505a6343f50f851d2e.vtt
	--- Downloaded subtitle a1477da82f45e776b7f889b67358e761.vtt
	--- Extracted thumbnail d1a7cda055c1a9f33887b53978371514.png
	--- Downloaded de498249b8d4395a4ef9db17ec02dc91.png


...

	Got file diff for 8 out of 8 files

Uploading 1 new file(s) to Kolibri Studio...


Uploading files...


	Uploaded 48d6b7e052f0f62ded3b6c149d10a4fc.json (1/1) 

Creating tree on Kolibri Studio...
   Creating channel Youtube subtitles downloading chef


Creating channel...


	Preparing fields...
(0 of 1 uploaded)    Processing Youtube subtitles downloading chef (ChannelNode)
   All nodes were created successfully.
Upload time: 2.453117s


DONE: Channel created at https://contentworkshop.learningequality.org/channels/94cae9e3b077507eaf642602c9059e70/edit

